## 神经网络卷积

In [2]:
#!pip install tensorboard
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.nn import Conv2d
#from torch.utils import DataLoader
from torch.utils.data import _utils
# tensorboard 导包
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#=================================================
# 数据集
# dataset = torchvision.datasets.CIFAR10(root='../dataset/data',train=False,transform=trans,
#                                      download=True)
# 载入数据
# dataloader = torch.utils.data.DataLoader(dataset,batch_size=64)

# 定义神经网络
class net(nn.Module):
    def __init__(self):
        super(net,self).__init__()
        # 一次卷积
        self.conv1 = Conv2d(in_channels=3,out_channels=3,kernel_size=3,stride=1,padding=0)
        
    def forward(self,x):
        x = self.conv1(x)
        return x

my_net = net()
# step=0
# for data in dataloader:
#     # 从data中获取图像和验证数据
#     imgs,tragets = data
#     # 输入:torch.Size([64, 3, 32, 32])
# #     print(imgs.shape)
#     writer.add_images('input',imgs,step)
#     output = my_net(imgs)
# #     print(output.shape)
#     # 输出：torch.Size([64, 6, 30, 30])
#     output = torch.reshape(output,(-1,3,30,30))
#     # print(output.shape)
#     writer.add_images('outputs',output,step)
#     step += 1
# 通过tensorBord来显示
#=================================================

# 转换数据类型为张量
trans = transforms.Compose([
                    transforms.ToTensor(),
                   # transforms.Resize()

                    ])

img_path = '../image/dog.png'

img_row = Image.open(img_path).convert('RGB')

img = trans(img_row)

writer = SummaryWriter('../log/conv')

# 写原始数据
writer.add_image('row picture',img)
print(img.shape)

# 卷积后的数据
output = my_net(img)
output = output.reshape(3,152,-1) # torch.Size([3, 152, 406])
print(output.shape)

#writer.add_image('conv picture',output)
    
print('img has finish convd')
    

torch.Size([3, 154, 205])
torch.Size([3, 152, 203])
img has finish convd


## 池化层
池化层有最大/平均池化层

输入图像和池化核进行覆盖，选择覆盖的最大值，当输入图像与池化核不能完全覆盖时，观察`ceil_mode`
- True:保留未覆盖中的最大值,输出图像大
- False：不保留,输出图像小
计算公式

$Input:(N,C,H_{in},W_{in})$:批次,通道,输入高度,输入宽度

$Output:(N,C,H_{out},W_{out})$:批次,通道,输出高度,输出宽度

$$H_{out}=[\frac{H_{in}+2*padding[0]-dilation[0]*(kernel\_size[0]-1)-1}{stride[0]}+1] $$
$$W_{out}=[\frac{w_{in}+2*padding[1]-dilation[1]*(kernel\_size[1]-1)-1}{stride[0]}+1] $$

最大池化作用：在保留数据特征的前提下，降低数据量


In [3]:
img = Image.open(img_path)
img_row = trans(img)


class nn_pool(nn.Module):
    def __init__(self):
        super(nn_pool,self).__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2)
        self.avgpool = nn.AvgPool2d(kernel_size=3,stride=2)
    def forward(self,x):
        output_max,output_avg = self.maxpool(x),self.avgpool(x)
        return output_max,output_avg

my_net = nn_pool()

output_max,output_avg = my_net(img_row)

writer.add_image('maxpool',output_max)
writer.add_image('avgpool',output_avg)
print('finish pool')

finish pool


In [4]:
# 输入图像,且要指定数据类型
input = torch.tensor([[1,2,0,3,1],
                      [0,1,2,3,1],
                      [1,2,1,0,0],
                      [5,2,3,1,1],
                      [2,1,0,1,1]],dtype=torch.float32)
# batch_size 自适应,1通道,5*5大小
input = torch.reshape(input,(-1,1,5,5))
# print(input.shape)

dataset = torchvision.datasets.CIFAR10('../dataset/data/',download=True,transform=trans,train=False)

dataloader = torch.utils.data.DataLoader(dataset,batch_size=64)


writer = SummaryWriter('../logs/maxpool')

class nn_pool(nn.Module):
    def __init__(self):
        super(nn_pool,self).__init__()
        # 最大池化，保留未覆盖的最大值
        self.maxpool_True = nn.MaxPool2d(kernel_size=3,ceil_mode=True)
        self.maxpool_False = nn.MaxPool2d(kernel_size=3,ceil_mode=False)
    def forward(self,x):
        output_True = self.maxpool_True(x)
        output_False = self.maxpool_False(x)
        return output_True,output_False

self_net = nn_pool()

output_ceil_True,output_ceil_False=self_net(input)
print('ceil mode为Ture:\n',output_ceil_True,'\nceil mode为False:\n',output_ceil_False)

step = 0 
for data in dataset:
    imgs,trage = data
    # 输入尺寸大小
   # print(imgs.shape)
    imgs = torch.reshape(imgs,(-1,3,32,32))
    print(imgs.shape)
    writer.add_images('input',imgs,step)
    output_true,output_false = self_net(imgs)
    writer.add_images('output_true',output_true,step)
    writer.add_images('output_false',output_false,step)
    # 输出尺寸大小
   # print(output_false.shape)
    step += 1

Files already downloaded and verified
ceil mode为Ture:
 tensor([[[[2., 3.],
          [5., 1.]]]]) 
ceil mode为False:
 tensor([[[[2.]]]])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])
torch.Size([1, 3, 32, 32])


## 非线性激活

 为神经网络引入非线性特征，提高模型的泛化特征

In [5]:
input = torch.tensor([[1,-0.5],
                      [-1,3]])

input = torch.reshape(input,(-1,1,2,2))
print(input)


# 引入数据集
dataset = torchvision.datasets.CIFAR10('../dataset/data/',download=True,transform=trans,train=False)

dataloader = torch.utils.data.DataLoader(dataset,batch_size=64)

writer = SummaryWriter('../log/sigmoid')

# 定义网络模型
class my_relu(nn.Module):
    def __init__(self):
        super(my_relu,self).__init__()
        # 非线性激活函数
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        output_relu = self.relu(x)
        output_sigmoid = self.sigmoid(x)
        return output_relu,output_sigmoid
    
my_no_linear = my_relu()

output_relu,my_sigmoid = my_no_linear(input)
print('relu:\n',output_relu,'\nsigoid:\n',my_sigmoid)

step = 0
for data in dataloader:
    imgs,targets = data
    
    writer.add_images('input',imgs,step)
    _,output=my_no_linear(imgs)
    writer.add_images('output',output,step)
    
    step += 1


tensor([[[[ 1.0000, -0.5000],
          [-1.0000,  3.0000]]]])
Files already downloaded and verified
relu:
 tensor([[[[1., 0.],
          [0., 3.]]]]) 
sigoid:
 tensor([[[[0.7311, 0.3775],
          [0.2689, 0.9526]]]])


## 其他网络层
线性层

In [11]:
class my_line(nn.Module):
    def __init__(self):
        super(my_line,self).__init__()
        # 196608 是根据输入图片的特征设置
        self.line = nn.Linear(196608,10)
    def forward(self,input):
        output = self.line(input)
        return output
Line = my_line()
    
for data in dataloader:
    imgs,targets = data
    print(imgs.shape)
    # 将输入更改为要求的格式，作为input
    output = torch.reshape(imgs,(1,1,1,-1))
    # output = torch.flatten(imgs)
    #print(output.shape)
    # 输出 torch.Size([1, 1, 1, 10])
    output = Line(output)
    #print(output.shape)
    

torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 3

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x49152 and 196608x8)

## 搭建网络模型

In [ ]:
class net(nn.Module):
    def __init__(self,class_indient=10):
        super(net,self).__init__()
        
        # in_channel,out_channel,kernel_size,stride,padding
        ''' 未使用Sequential
        self.conv1 = nn.Conv2d(3, 32,5,padding=2)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32,32,5,padding=2)
        self.maxpool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32,64,5,padding=2)
        self.maxpool3 = nn.MaxPool2d(2)
        self.flatten = nn.Flatten()
        self.line1 = nn.Linear(1024,64)
        self.line2 = nn.Linear(64,10)
        '''
#         self.model1 = nn.Sequential(
#             nn.Conv2d(3, 32,5,padding=2),
#             nn.MaxPool2d(2),
#             nn.Conv2d(32,32,5,padding=2),
#             nn.MaxPool2d(2),
#             nn.Conv2d(32,64,5,padding=2),
#             nn.MaxPool2d(2),
#             nn.Flatten(),
#             nn.Linear(1024,64),
#             nn.Linear(64,10)
#         )
        
    
        self.model1 = nn.Sequential(
            #  in_channel：1 out_channel:6
            nn.Conv2d(1,6,5,padding=2),nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2,stride=2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5,120),nn.Sigmoid(),
            nn.Linear(120,84),nn.Sigmoid(),
            nn.Linear(84,class_indient))
    
    def forward(self,x):
        '''
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.flatten(x)
        x = self.line1(x)
        x = self.line2(x)
        '''
        x = self.model1(x)
        return x
    
# 通过输出数据的类型来查看网络的正确性

Net = net(class_indient=10)

#print('网络结构：\n',Net)
input  = torch.ones(1,1,28,28)
#output = Net(input)
##########################

# 迭代输出每层的网络和尺寸
for layer in Net.model1:
    input = layer(input)
    print(layer.__class__.__name__,'output shape:\t',input.shape)
# Conv2d output shape:	 torch.Size([1, 6, 28, 28])
# Sigmoid output shape:	 torch.Size([1, 6, 28, 28])
# AvgPool2d output shape:	 torch.Size([1, 6, 14, 14])
# Conv2d output shape:	 torch.Size([1, 16, 10, 10])
# Sigmoid output shape:	 torch.Size([1, 16, 10, 10])
# AvgPool2d output shape:	 torch.Size([1, 16, 5, 5])
# Flatten output shape:	 torch.Size([1, 400])
# Linear output shape:	 torch.Size([1, 120])
# Sigmoid output shape:	 torch.Size([1, 120])
# Linear output shape:	 torch.Size([1, 84])
# Sigmoid output shape:	 torch.Size([1, 84])
# Linear output shape:	 torch.Size([1, 10])

    
    
##########################
# 测试数据
#output.shape

writer = SummaryWriter('../log/log_seq')
# tensorboard写网络结构(可视化显示)
writer.add_graph(Net,input)
writer.close()

In [ ]:
!tensorboard --logdir=../log/log_seq --port=7777

## 损失函数和反向传播

损失函数作用：
1. 计算实际输出和目标输出之间的误差
2. 为更新输出提供依据(反向传播)

In [9]:
inputs = torch.tensor([1,2,3],dtype=torch.float32)
targets = torch.tensor([1,2,5],dtype=torch.float32)

# 更改形状

inputs = torch.reshape(inputs,(1,1,1,3))
targets = torch.reshape(targets,(1,1,1,3))

# 损失函数
loss = nn.L1Loss()
loss_mse = nn.MSELoss()

result = loss(inputs,targets)
result_mse = loss_mse(inputs,targets)
print("损失结果：",result)
print("平方差损失结果：",result_mse)

# 交叉熵
x = torch.tensor([0.1,0.2,0.3])
y = torch.tensor([1])
x = torch.reshape(x,(1,3))
loss_cross = nn.CrossEntropyLoss()
result_cross = loss_cross(x,y)
print('交叉熵：',result_cross)


损失结果： tensor(0.6667)
平方差损失结果： tensor(1.3333)
交叉熵： tensor(1.1019)


In [10]:
# 数据集
dataset = torchvision.datasets.CIFAR10('../dataset/data',download=True,transform= torchvision.transforms.ToTensor(),
                                      train = False)
# 载入数据
dataloader = torch.utils.data.DataLoader(dataset,batch_size=1)

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            # dilation 默认为 1
            nn.Conv2d(3,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,padding=2),
            nn.MaxPool2d(2),
            # 展平
            # start_dim:0
            # end_dim:2
            # 将数据展开(原本是维度不一致不能进行矩阵乘法)
            nn.Flatten(0,2),
            nn.Linear(1024,64),
            nn.Linear(64,10)        
        )

    def forward(self,x):
        x = self.model1(x)
        return x



net = Net()
# 交叉熵
loss = nn.CrossEntropyLoss()
for data in dataset:
    imgs ,targets = data
    # targets 类型更改为 tensor 类型
    targets = torch.tensor(targets)
    output = net(imgs)
    # 误差
    loss_result = loss(output,targets)
    #print(loss_rsult)
    #反向传播
    # loss_result.backward()


Files already downloaded and verified


## 优化器

优化器根据梯度来选择合适的参数

In [138]:
# 数据集
dataset = torchvision.datasets.CIFAR10('../dataset/data',download=True,transform= torchvision.transforms.ToTensor(),
                                      train = False)
# 载入数据
dataloader = torch.utils.data.DataLoader(dataset,batch_size=64)
# 网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            # dilation 默认为 1
            nn.Conv2d(3,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,5,padding=2),
            nn.MaxPool2d(2),
            # 展平
            # start_dim:0
            # end_dim:2
            # 将数据展开(原本是维度不一致不能进行矩阵乘法)
            nn.Flatten(0,2),
            nn.Linear(1024,64),
            nn.Linear(64,10)        
        )

    def forward(self,x):
        x = self.model1(x)
        return x

net = Net()
# 交叉熵
loss = nn.CrossEntropyLoss()
# 优化器
optim = torch.optim.SGD(net.parameters(),lr=0.01)
# 多轮更新参数
for epoch in range(20):
    # 每轮都将损失置为 0
    runing_loss = 0.0
    print('epoch {}'.format(epoch+1))
    for data in dataset:
        imgs ,targets = data
        # targets 类型更改为 tensor 类型
        targets = torch.tensor(targets)
        output = net(imgs)
        # 误差
        loss_result = loss(output,targets)
        print(loss_result)
        # 梯度清零
        optim.zero_grad() 
        #反向传播
        loss_result.backward()
        # 参数调优
        optim.step()
        runing_loss = runing_loss + loss_result
    print("epoch:",epoch,"损失值",runing_loss)

Files already downloaded and verified
epoch 1
tensor(2.3393, grad_fn=<NllLossBackward0>)
tensor(2.3400, grad_fn=<NllLossBackward0>)
tensor(2.2620, grad_fn=<NllLossBackward0>)
tensor(2.3417, grad_fn=<NllLossBackward0>)
tensor(2.3289, grad_fn=<NllLossBackward0>)
tensor(2.2728, grad_fn=<NllLossBackward0>)
tensor(2.2800, grad_fn=<NllLossBackward0>)
tensor(2.2299, grad_fn=<NllLossBackward0>)
tensor(2.2917, grad_fn=<NllLossBackward0>)
tensor(2.3014, grad_fn=<NllLossBackward0>)
tensor(2.2678, grad_fn=<NllLossBackward0>)
tensor(2.3872, grad_fn=<NllLossBackward0>)
tensor(2.2688, grad_fn=<NllLossBackward0>)
tensor(2.4848, grad_fn=<NllLossBackward0>)
tensor(2.3506, grad_fn=<NllLossBackward0>)
tensor(2.2880, grad_fn=<NllLossBackward0>)
tensor(2.2476, grad_fn=<NllLossBackward0>)
tensor(2.4619, grad_fn=<NllLossBackward0>)
tensor(2.2191, grad_fn=<NllLossBackward0>)
tensor(2.2531, grad_fn=<NllLossBackward0>)
tensor(2.4065, grad_fn=<NllLossBackward0>)
tensor(2.2490, grad_fn=<NllLossBackward0>)
tensor(2

tensor(2.4397, grad_fn=<NllLossBackward0>)
tensor(2.2161, grad_fn=<NllLossBackward0>)
tensor(2.1023, grad_fn=<NllLossBackward0>)
tensor(1.7911, grad_fn=<NllLossBackward0>)
tensor(2.4166, grad_fn=<NllLossBackward0>)
tensor(1.6945, grad_fn=<NllLossBackward0>)
tensor(2.3925, grad_fn=<NllLossBackward0>)
tensor(2.6531, grad_fn=<NllLossBackward0>)
tensor(1.7219, grad_fn=<NllLossBackward0>)
tensor(2.4005, grad_fn=<NllLossBackward0>)
tensor(2.4365, grad_fn=<NllLossBackward0>)
tensor(2.5654, grad_fn=<NllLossBackward0>)
tensor(2.2521, grad_fn=<NllLossBackward0>)
tensor(2.2645, grad_fn=<NllLossBackward0>)
tensor(2.3598, grad_fn=<NllLossBackward0>)
tensor(2.4628, grad_fn=<NllLossBackward0>)
tensor(2.3321, grad_fn=<NllLossBackward0>)
tensor(2.7086, grad_fn=<NllLossBackward0>)
tensor(2.1971, grad_fn=<NllLossBackward0>)
tensor(2.2977, grad_fn=<NllLossBackward0>)
tensor(1.9004, grad_fn=<NllLossBackward0>)
tensor(2.2438, grad_fn=<NllLossBackward0>)
tensor(2.2751, grad_fn=<NllLossBackward0>)
tensor(2.27

KeyboardInterrupt: 

## 现有网络模型的修改

In [12]:
# ImageNet的使用(太大，无法下载)
# 需要安装 scipy
# ! pip install scipy

# dataset = torchvision.datasets.ImageNet("../data",split='train',download=True,
#                                       transform=trans)
'''
vgg16 模型的修改
'''
vgg16_true = torchvision.models.vgg16(pretrained=True)
vgg16_false = torchvision.models.vgg16(pretrained=False)

# 在数据集上已经训练好的网络
# print(vgg16_true)


# vgg16_true.add_module('add_linear',nn.Linear(1000,10))
# print(vgg16_true)

# 将线形层加入 classifier 中
# vgg16_true.classifier.add_module('add_linear',nn.Linear(1000,10))
# print(vgg16_true)

# 修改网络
vgg16_false.classifier[6]=nn.Linear(4096,10)
print(vgg16_false)
# 更改回默认
vgg16_false.classifier[6]=nn.Linear(4096,1000)


/home/yy/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/yy/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/yy/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 网络模型保存和读取

In [13]:
# 保存方式一(模型结构+模型参数)
torch.save(vgg16_false,'vgg16_method1.pth')

# 加载保持方式一的模型
model = torch.load('vgg16_method1.pth')
# 显示模型
print('method1:\n',model)



# 保存方式二(字典形式：模型参数)
torch.save(vgg16_false.state_dict(),'vgg16_method2.pth')
# 加载 保持方式二的模型
vgg16 = torchvision.models.vgg16(pretrained=False)
vgg16.load_state_dict(torch.load("vgg16_method2.pth"))
# model = torch.load('vgg16_method2.pth')
# 显示模型
print("method2:\n",vgg16)


method1:
 VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, 

## 

## 完整的训练流程

In [76]:
import torchvision
import time
trans = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.CIFAR10('../dataset/data',train=True,transform=trans,
                                         download=True)

test_data = torchvision.datasets.CIFAR10('../dataset/data',train=False,transform=trans,
                                         download=True)

# 训练数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print('训练数据集的长度为:{}'.format(train_data_size))
print('测试数据集的长度为:{}'.format(test_data_size))

# 利用 DataLoader 来加载数据集
train_dataloader = torch.utils.data.DataLoader(train_data,batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=64)


# 搭建神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64, 5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10) )
    def forward(self,x):
        x = self.model1(x)
        return x
    
net = Net()
# 验证模型正确性
input = torch.ones(64,3,32,32)
output = net(input)
# print(output.shape)

# 损失函数
loss_fn = nn.CrossEntropyLoss()

# 优化器
learning_rate = 0.01
# 1e-2 = 1*10^(-2)=0.01
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate)

# 设置训练网络的参数

# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加 tensorboard
writer = SummaryWriter('../log/logs_train')


for i in range(epoch):
    tst = time.time() 
    print("-------第 {} 轮训练开始-------".format(i+1))
    
    # 训练开始
    for data in train_dataloader:
        img,targets = data
        output = net(img)
        loss = loss_fn(output,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step = total_train_step + 1
        # 简略输出
        if total_train_step % 200 == 0:
            # item==>转换为具体的值
            print('训练次数：{},loss:{}'.format(total_train_step,loss.item()))
            writer.add_scalar('train_loss',loss.item(),total_train_step)
        
    # 测试步骤
    total_test_loss = 0
    total_accury = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            output = net(imgs)
            loss = loss_fn(output,targets)
            total_test_loss = total_test_loss + loss.item()
            # 正确率的比较
            accuary =(output.argmax(1) == targets).sum()
            total_accury = total_accury + accuary
    print('整体测试集上的 Loss：{}'.format(total_test_loss))
    print('整体测试集上的正确率：{}'.format(total_accury/test_data_size))
    # tensorboard 可视化
    writer.add_scalar('test_loss',total_test_loss,total_test_step)
    writer.add_scalar('test_accuarcy',total_accury/test_data_size,total_test_step)
    total_test_step = total_test_step + 1
    
    # 保存每轮训练的结果
    torch.save(net,'../net_model/net_{}.pth'.format(i))
    # torch.save(net.state_dict,"../model/net_{}.pth")
    print('模型已保存')
    print('第 {} 轮的运行时间 {} ms'.format(i+1,(time.time()-tst)))

writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为:50000
测试数据集的长度为:10000
-------第 1 轮训练开始-------
训练次数：200,loss:2.2879884243011475
训练次数：400,loss:2.200949192047119
训练次数：600,loss:2.040105104446411
整体测试集上的 Loss：320.6351805925369
整体测试集上的正确率：0.2621000111103058
模型已保存
第 1 轮的运行时间 11.653879880905151 ms
-------第 2 轮训练开始-------
训练次数：800,loss:1.9051201343536377
训练次数：1000,loss:1.954735279083252
训练次数：1200,loss:1.7706853151321411
训练次数：1400,loss:1.7771344184875488
整体测试集上的 Loss：309.75282990932465
整体测试集上的正确率：0.3003000020980835
模型已保存
第 2 轮的运行时间 10.40507173538208 ms
-------第 3 轮训练开始-------
训练次数：1600,loss:1.7924138307571411
训练次数：1800,loss:1.9302079677581787
训练次数：2000,loss:1.9304711818695068
训练次数：2200,loss:1.4683151245117188
整体测试集上的 Loss：276.11497139930725
整体测试集上的正确率：0.3652999997138977
模型已保存
第 3 轮的运行时间 10.530328035354614 ms
-------第 4 轮训练开始-------
训练次数：2400,loss:1.7496556043624878
训练次数：2600,loss:1.6045993566513062
训练次数：2800,loss:1.4968572854995728
训练次数：3000,loss:1.382225155

## 模型的二分类

In [15]:
# 模型的二分类的
outputs = torch.tensor([[0.1,0.2],
                       [0.03,0.4]])
# 1:横向 0：纵向
print("横向：",outputs.argmax(1))
print("纵向：",outputs.argmax(0))

preds = outputs.argmax(1) 
# 标准
targets = torch.tensor([0,1])

print('整体一致数量:',(targets == preds).sum().item())

横向： tensor([1, 1])
纵向： tensor([0, 1])
整体一致数量: 1


## 使用 GPU训练

In [84]:
import time
# 第一种 GPU 训练方式
import torchvision

trans = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.CIFAR10('../dataset/data',train=True,transform=trans,
                                         download=True)

test_data = torchvision.datasets.CIFAR10('../dataset/data',train=False,transform=trans,
                                         download=True)

# 训练数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print('训练数据集的长度为:{}'.format(train_data_size))
print('测试数据集的长度为:{}'.format(test_data_size))

# 利用 DataLoader 来加载数据集
train_dataloader = torch.utils.data.DataLoader(train_data,batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=64)

# 类别索引
# {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
class_index = train_data.class_to_idx
# 更改 类别和值的位置
# {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}
class_dict = dict((val,key) for key,val in class_index.items())

#print(class_index)

# 搭建神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64, 5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10) )
    def forward(self,x):
        x = self.model1(x)
        return x
    
net = Net()
# GPU
if torch.cuda.is_available():
    net = net.cuda() 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
# 验证模型正确性
# input = torch.ones(64,3,32,32)
# output = net(input.cuda())
# print(output.shape)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    loss_fn = loss_fn.cuda()

# 优化器
learning_rate = 0.01
# 1e-2 = 1*10^(-2)=0.01
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate)

# 设置训练网络的参数

# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加 tensorboard
writer = SummaryWriter('../log/log_gpu/')

for i in range(epoch):
    tst = time.time()
    print("-------第 {} 轮训练开始-------".format(i+1))
    
    # 训练开始
    net.train()
    for data in train_dataloader:
        img,targets = data
        if torch.cuda.is_available():
            img = img.cuda()
            targets =targets.cuda()
        output = net(img)
        loss = loss_fn(output,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step = total_train_step + 1
        # 简略输出
        if total_train_step % 200 == 0:
            # item==>转换为具体的值
            print('训练次数：{},loss:{}'.format(total_train_step,loss.item()))
            writer.add_scalar('train_loss',loss.item(),total_train_step)
        
    # 测试步骤
    net.eval()
    total_test_loss = 0
    total_accury = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            # 显卡可用使用 GPU 加速
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                targets =targets.cuda()
            outputs = net(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss = total_test_loss + loss.item()
            # 正确率的比较
            accuary =(outputs.argmax(1) == targets).sum()
            total_accury = total_accury + accuary
    print('整体测试集上的 Loss：{}'.format(total_test_loss))
    print('整体测试集上的正确率：{}'.format(total_accury/test_data_size))
    # tensorboard 可视化
    writer.add_scalar('test_loss',total_test_loss,total_test_step)
    writer.add_scalar('test_accuarcy',total_accury/test_data_size,total_test_step)
    total_test_step = total_test_step + 1
    
    # 保存每轮训练的结果
    torch.save(net,'../net_model/net_{}.pth'.format(i))
    # torch.save(net.state_dict,"../model/net_{}.pth")
    print('模型已保存')
    print('第{}轮运行时间：{} ms'.format(i+1,time.time()-tst))

writer.close()

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为:50000
测试数据集的长度为:10000
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
{0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}


SyntaxError: 'break' outside loop (4242305597.py, line 32)

In [79]:
import time
# 第二种 GPU 训练方式
''' .to(device)
    Device = torch.device('cpu')
    # 多个显卡可以指定显卡
    torch.device('cuda')
    torch.device('cuda:0')
'''
import torchvision

# 定义训练设备
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)


trans = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.CIFAR10('../dataset/data',train=True,transform=trans,
                                         download=True)

test_data = torchvision.datasets.CIFAR10('../dataset/data',train=False,transform=trans,
                                         download=True)

# 训练数据集长度
train_data_size = len(train_data)
test_data_size = len(test_data)

print('训练数据集的长度为:{}'.format(train_data_size))
print('测试数据集的长度为:{}'.format(test_data_size))

# 利用 DataLoader 来加载数据集
train_dataloader = torch.utils.data.DataLoader(train_data,batch_size=64)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size=64)
# 搭建神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64, 5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10) )
    def forward(self,x):
        x = self.model1(x)
        return x
    
net = Net()
# GPU 一
# if torch.cuda.is_available():
#     net = net.cuda() 

# CPU 二
net = net.to(device)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
# 验证模型正确性
# input = torch.ones(64,3,32,32)
# output = net(input.cuda())
# print(output.shape)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# if torch.cuda.is_available():
#     loss_fn = loss_fn.cuda()

loss_fn = loss_fn.to(device)
# 优化器
learning_rate = 0.01
# 1e-2 = 1*10^(-2)=0.01
optimizer = torch.optim.SGD(net.parameters(),lr=learning_rate)

# 设置训练网络的参数

# 记录训练次数
total_train_step = 0
# 记录测试次数
total_test_step = 0
# 训练的轮数
epoch = 10

# 添加 tensorboard
writer = SummaryWriter('../log/log_gpu/')

for i in range(epoch):
    tst = time.time()
    print("-------第 {} 轮训练开始-------".format(i+1))
    
    # 训练开始
    net.train()
    for data in train_dataloader:
        img,targets = data
#         if torch.cuda.is_available():
#             img = img.cuda()
#             targets =targets.cuda()
        img = img.to(device)
        targets = targets.to(device)
        output = net(img)
        loss = loss_fn(output,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step = total_train_step + 1
        # 简略输出
        if total_train_step % 200 == 0:
            # item==>转换为具体的值
            print('训练次数：{},loss:{}'.format(total_train_step,loss.item()))
            writer.add_scalar('train_loss',loss.item(),total_train_step)
        
    # 测试步骤
    net.eval()
    total_test_loss = 0
    total_accury = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            # 显卡可用使用 GPU 加速
#             if torch.cuda.is_available():
#                 imgs = imgs.cuda()
#                 targets =targets.cuda()
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = net(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss = total_test_loss + loss.item()
            # 正确率的比较
            accuary =(outputs.argmax(1) == targets).sum()
            total_accury = total_accury + accuary
    print('整体测试集上的 Loss：{}'.format(total_test_loss))
    print('整体测试集上的正确率：{}'.format(total_accury/test_data_size))
    # tensorboard 可视化
    writer.add_scalar('test_loss',total_test_loss,total_test_step)
    writer.add_scalar('test_accuarcy',total_accury/test_data_size,total_test_step)
    total_test_step = total_test_step + 1
    
    # 保存每轮训练的结果
    torch.save(net,'../net_model/net_{}.pth'.format(i))
    # torch.save(net.state_dict,"../model/net_{}.pth")
    print('模型已保存')
    print('第{}轮运行时间：{} ms'.format(i+1,time.time()-tst))
    

writer.close()

cuda:0
Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为:50000
测试数据集的长度为:10000
-------第 1 轮训练开始-------
训练次数：200,loss:2.2709596157073975
训练次数：400,loss:2.136503219604492
训练次数：600,loss:2.007432699203491
整体测试集上的 Loss：317.60504698753357
整体测试集上的正确率：0.2797999978065491
模型已保存
第1轮运行时间：7.000423431396484 ms
-------第 2 轮训练开始-------
训练次数：800,loss:1.8889195919036865
训练次数：1000,loss:1.930546760559082
训练次数：1200,loss:1.702648639678955
训练次数：1400,loss:1.7428433895111084
整体测试集上的 Loss：301.1404789686203
整体测试集上的正确率：0.31619998812675476
模型已保存
第2轮运行时间：6.925467252731323 ms
-------第 3 轮训练开始-------
训练次数：1600,loss:1.7271140813827515
训练次数：1800,loss:1.9148608446121216
训练次数：2000,loss:1.9648995399475098
训练次数：2200,loss:1.4693820476531982
整体测试集上的 Loss：267.54984760284424
整体测试集上的正确率：0.3870999813079834
模型已保存
第3轮运行时间：6.958582162857056 ms
-------第 4 轮训练开始-------
训练次数：2400,loss:1.7492176294326782
训练次数：2600,loss:1.5839513540267944
训练次数：2800,loss:1.5008862018585205
训练次数：3000,loss:1.3549327850341

In [71]:
device = torch.device('cuda',0x00)
# 显卡数量
gpu_num = torch.cuda.device_count()
print(gpu_num)

6


## 完整模型测试

In [140]:
# GPU训练的模型,想在 CPU 上运行需要指定(map_loaction)
from PIL import Image
import time
# 测试的图片路径
image_path = '../image/dog.png'

image = Image.open(image_path)
# print("原通道:",len(image.split()))
# PNG 格式是四通道，除了 RGB三通道还有一个透明通道，通过转变保留原通道数
image = image.convert('RGB')
# print("现通道:",len(image.split()))
print(image)
# 更改类型(图片大小,类型都需要更改)
transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),
                                            trans])

image = transforms(image)
# 假设数据进行预测
#image2 = torch.randn_like(image)
#images = torch.stack([image, image2])

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,32,5,1,2),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64, 5,1,2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4,64),
            nn.Linear(64,10) )
    def forward(self,x):
        x = self.model1(x)
        return x
net = Net()
# 加载模型(GPU 训练，cpu加载)
model = torch.load('../net_model/net_2.pth',map_location=torch.device('cpu'))   
print(model)
image = torch.reshape(image,(1,3,32,32))
model.eval()
# 类别
classes =('plane','car','bird','cat',
         'deer','dog','forg','horse','ship','truck')
with torch.no_grad():
    # print(image.shape)
    # 预留出batch_size的位置
    #image = torch.unsqueeze(image,dim=0)
    output = model(image)
# output = net(image)
# print("output 的形状: ", output.shape)
print(classes)
print("output 的概率: ", torch.softmax(output, dim=-1))
# 测试输出的类别
print("类别：",classes[output.argmax(-1).item()])

<PIL.Image.Image image mode=RGB size=205x154 at 0x7FB843363DF0>
Net(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)
('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'forg', 'horse', 'ship', 'truck')
output 的概率:  tensor([[0.0413, 0.0310, 0.1265, 0.1596, 0.1254, 0.2411, 0.1112, 0.1474, 0.0067,
         0.0098]])
类别： dog
